##### Copyright 2019 The TensorFlow Authors.

In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [2]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 使用 Keras 和 Tensorflow Hub 对电影评论进行文本分类

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://tensorflow.google.cn/tutorials/keras/text_classification_with_hub"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/tutorials/keras/text_classification_with_hub.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a>   </td>
  <td><a href="https://tfhub.dev/s?module-type=text-embedding"><img src="https://tensorflow.google.cn/images/hub_logo_32px.png">查看 TF Hub 模型</a></td>
</table>

此笔记本（notebook）使用评论文本将影评分为*积极（positive）*或*消极（nagetive）*两类。这是一个*二元（binary）*或者二分类问题，一种重要且应用广泛的机器学习问题。

本教程演示了使用 Tensorflow Hub 和 Keras 进行迁移学习的基本应用。

我们将使用包含 [Internet Movie Database](https://www.imdb.com/) 中的 50,000 条电影评论文本的 [IMDB 数据集](https://tensorflow.google.cn/api_docs/python/tf/keras/datasets/imdb)。先将这些评论分为两组，其中 25,000 条用于训练，另外 25,000 条用于测试。训练组和测试组是*均衡的*，也就是说其中包含相等数量的正面评价和负面评价。

此笔记本使用 [`tf.keras`](https://tensorflow.google.cn/guide/keras)（一个在 TensorFlow 中用于构建和训练模型的高级 API）和 [`tensorflow_hub`](https://tensorflow.google.cn/hub)（一个用于在一行代码中从 [TFHub](https://tfhub.dev) 加载训练模型的库）。有关使用 `tf.keras` 的更高级文本分类教程，请参阅 [MLCC 文本分类指南](https://developers.google.com/machine-learning/guides/text-classification/)。

In [3]:
# !pip install tensorflow-hub
# !pip install tensorflow-datasets

In [4]:
import numpy as np

import tensorflow as tf

# !pip install tensorflow-hub
# !pip install tfds-nightly
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

Version:  2.8.0-dev20211203
Eager mode:  True
Hub version:  0.12.0
GPU is NOT AVAILABLE


## 下载 IMDB 数据集

IMDB数据集可以在 [Tensorflow 数据集](https://github.com/tensorflow/datasets)处获取。以下代码将 IMDB 数据集下载至您的机器（或 colab 运行时环境）中：

In [5]:
# Split the training set into 60% and 40% to end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /Users/huliqun/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIQHGVS/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /Users/huliqun/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteIQHGVS/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

2021-12-09 17:01:45.102343: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Dataset imdb_reviews downloaded and prepared to /Users/huliqun/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## 探索数据

我们花一点时间来了解数据的格式。每个样本都是一个代表电影评论的句子和一个相应的标签。句子未经过任何预处理。标签是一个整数值（0 或 1），其中 0 表示负面评价，而 1 表示正面评价。

我们来打印下前十个样本。

In [6]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

我们再打印下前十个标签。

In [7]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

## 构建模型

神经网络由堆叠的层来构建，这需要从三个主要方面来进行体系结构决策：

- 如何表示文本？
- 模型里有多少层？
- 每个层里有多少*隐层单元（hidden units）*？

本示例中，输入数据由句子组成。预测的标签为 0 或 1。

表示文本的一种方式是将句子转换为嵌入向量。使用一个预训练文本嵌入向量作为首层，这将具有三个优点：

- 不必担心文本预处理
- 可以从迁移学习中受益
- 嵌入具有固定长度，更易于处理

在本示例中，您使用来自 [TensorFlow Hub](https://tfhub.dev) 的 **预训练文本嵌入向量模型**，名称为 [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)。

本教程中还可以使用来自 TFHub 的许多其他预训练文本嵌入向量：

- [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) - 基于与 [google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2) 相同的数据并使用相同的 NNLM 架构进行训练，但具有更大的嵌入向量维度。更大维度的嵌入向量可以改进您的任务，但可能需要更长的时间来训练您的模型。
- [google/nnlm-en-dim128-with-normalization/2](https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2) - 与 [google/nnlm-en-dim128/2](https://tfhub.dev/google/nnlm-en-dim128/2) 相同，但具有额外的文本归一化，例如移除标点符号。如果您的任务中的文本包含附加字符或标点符号，这会有所帮助。
- [google/universal-sentence-encoder/4](https://tfhub.dev/google/universal-sentence-encoder/4) - 一个可产生 512 维嵌入向量的更大模型，使用深度平均网络 (DAN) 编码器训练。

还有很多！在 TFHub 上查找更多[文本嵌入向量模型](https://tfhub.dev/s?module-type=text-embedding)。

让我们首先创建一个使用 Tensorflow Hub 模型嵌入（embed）语句的Keras层，并在几个输入样本中进行尝试。请注意无论输入文本的长度如何，嵌入（embeddings）输出的形状都是：`(num_examples, embedding_dimension)`。

In [8]:
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

现在让我们构建完整模型：

In [12]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense_4 (Dense)             (None, 16)                816       
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48,191,433
Trainable params: 48,191,433
Non-trainable params: 0
_________________________________________________________________


层按顺序堆叠以构建分类器：

1. 第一层是 TensorFlow Hub 层。此层使用预训练的 SaveModel 将句子映射到其嵌入向量。您使用的预训练文本嵌入向量模型 ([google/nnlm-en-dim50/2](https://tfhub.dev/google/nnlm-en-dim50/2)) 可将句子拆分为词例，嵌入每个词例，然后组合嵌入向量。生成的维度是：`(num_examples, embedding_dimension)`。对于此 NNLM 模型，`embedding_dimension` 是 50。
2. 该定长输出向量通过一个有 16 个隐层单元的全连接层（`Dense`）进行管道传输。
3. 最后一层与单个输出结点紧密相连。使用 `Sigmoid` 激活函数，其函数值为介于 0 与 1 之间的浮点数，表示概率或置信水平。

让我们编译模型。

### 损失函数与优化器

一个模型需要一个损失函数和一个优化器来训练。由于这是一个二元分类问题，并且模型输出 logit（具有线性激活的单一单元层），因此，我们将使用 `binary_crossentropy` 损失函数。

这并非损失函数的唯一选择，例如，您还可以选择 `mean_squared_error`。但是，一般来说，`binary_crossentropy` 更适合处理概率问题，它可以测量概率分布之间的“距离”，或者在我们的用例中，是指真实分布与预测值之间的差距。

稍后，当您探索回归问题（例如，预测房屋价格）时，您将看到如何使用另一个称为均方误差的损失函数。

现在，配置模型来使用优化器和损失函数：

In [14]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## 训练模型

使用包含 512 个样本的 mini-batch 对模型进行 10 个周期的训练，也就是在 `x_train` 和 `y_train` 张量中对所有样本进行 10 次迭代。在训练时，监测模型在验证集的 10,000 个样本上的损失和准确率：

In [15]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 25s 741ms/step - loss: 0.6637 - accuracy: 0.5170 - val_loss: 0.6072 - val_accuracy: 0.5805
Epoch 2/10
30/30 [==============================] - 21s 681ms/step - loss: 0.5408 - accuracy: 0.6755 - val_loss: 0.5008 - val_accuracy: 0.7381
Epoch 3/10
30/30 [==============================] - 20s 652ms/step - loss: 0.4062 - accuracy: 0.8210 - val_loss: 0.4039 - val_accuracy: 0.8054
Epoch 4/10
30/30 [==============================] - 20s 663ms/step - loss: 0.2932 - accuracy: 0.8869 - val_loss: 0.3476 - val_accuracy: 0.8464
Epoch 5/10
30/30 [==============================] - 20s 653ms/step - loss: 0.2138 - accuracy: 0.9241 - val_loss: 0.3199 - val_accuracy: 0.8589
Epoch 6/10
30/30 [==============================] - 20s 652ms/step - loss: 0.1570 - accuracy: 0.9507 - val_loss: 0.3087 - val_accuracy: 0.8643
Epoch 7/10
30/30 [==============================] - 20s 649ms/step - loss: 0.1147 - accuracy: 0.9675 - val_loss: 0.3068 - val_accuracy: 0.8665

## 评估模型

我们来看下模型的表现如何。将返回两个值。损失值（loss）（一个表示误差的数字，值越低越好）与准确率（accuracy）。

In [16]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 6s - loss: 0.3548 - accuracy: 0.8528 - 6s/epoch - 124ms/step
loss: 0.355
accuracy: 0.853


这种十分朴素的方法得到了约 87% 的准确率（accuracy）。若采用更好的方法，模型的准确率应当接近 95%。

## 进一步阅读

- 有关处理字符串输入的更通用方式以及对训练过程中准确率和损失进度的更详细分析，请参阅[使用预处理文本的文本分类](./text_classification.ipynb)教程。
- 尝试更多使用来自 TFHub 的训练模型的[文本相关教程](https://tensorflow.google.cn/hub/tutorials#text-related-tutorials)。